In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
KEY = os.getenv("API_KEY")

from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
model = ChatOpenAI(api_key=KEY)

### Comma separated output parser

In [5]:
# import parser
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [6]:
# format instructions
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [9]:
reply = "red, blue, green"
output_parser.parse(reply)

['red', 'blue', 'green']

In [12]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [13]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [16]:
model_request = chat_prompt.format_prompt(request = "give me 5 characteristics of cats",
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [21]:
result = model.invoke(model_request)
output_parser.parse(result.content)

['Independent', 'agile', 'curious', 'aloof', 'playful']

### Output fixing parser

In [23]:
from langchain.output_parsers import DatetimeOutputParser
dt_output_parser = DatetimeOutputParser()

In [24]:
dt_output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0713-04-09T12:04:42.094158Z, 0255-09-01T02:27:28.775106Z, 0871-11-13T12:42:12.329269Z\n\nReturn ONLY this string, no other words!"

In [25]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

In [26]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [52]:
model_request = chat_prompt.format_prompt(request="what date was the 13th Amendment ratified?",
                          format_instructions=dt_output_parser.get_format_instructions()).to_messages()

In [53]:
result = model.invoke(model_request, temperature=0)
result.content

'1865-12-06T00:00:00.000000Z'

In [54]:
from langchain.output_parsers import OutputFixingParser

In [55]:
misformatted = result.content

In [62]:
new_parser = OutputFixingParser.from_llm(parser=dt_output_parser, llm=model)
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)